In [6]:
!pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from z3 import *

pessoas = ["Ana","Nuno","Pedro","Maria"]
gabs = [1,2,3]
x = {}
for p in pessoas:
    x[p] = {}
    for g in gabs:
        x[p][g] = Bool("%s,%d" % (p,g))

s = Solver()



# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
    s.add(Or([x[p][g] for g in gabs]))

# Cada pessoa ocupa no máximo um gabinete.
for p in pessoas:
    s.add(Sum([If(x[p][g], 1, 0) for g in gabs]) == 1)

# O Nuno e o Pedro não podem partilhar gabinete.
for g in gabs:
    s.add(Or(Not(x["Pedro"][g]), Not(x["Nuno"][g])))

# Se a Ana ficar sozinha num gabinete, então o Pedro também terá
# que ficar sozinho num gabinete.
#s.add(And((Sum([If(x["Ana"][g], 1, 0) for g in gabs]) == 1),(Sum([If(x["Pedro"][g], 1, 0) for g in gabs]) == 1))) #Não sei se está certa
for g in gabs:
    # A "Ana" está no gabinete "g" e mais ninguem está
    ana_alone = And(x["Ana"][g], Sum([If(x[p][g], 1, 0) for p in pessoas if p != "Ana"]) == 0)
    # O "Pedro" está sozinho em algum gabinete
    pedro_alone = Or([And(x["Pedro"][g2], Sum([If(x[p][g2], 1, 0) for p in pessoas if p != "Pedro"]) == 0) for g2 in gabs])
    # Se a "Ana" está sozinha então o "Pedro" têm de estar sozinho
    s.add(Implies(ana_alone, pedro_alone))

# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
for g in gabs:
    s.add(Sum([If(x[p][g], 1, 0) for p in pessoas]) <= 2)

s.push()

if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Ana fica no gabinete 1
Nuno fica no gabinete 1
Pedro fica no gabinete 3
Maria fica no gabinete 3


In [8]:
#Faça as alterações necessárias ao programa de modo a saber todas as distribuições possíveis

# Definindo a lista de soluções
solucoes = []

# Loop para encontrar todas as soluções
while s.check() == sat:
    m = s.model()
    
    # Armazena a solução atual
    solucao_atual = []
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                solucao_atual.append((p, g))
                print("%s fica no gabinete %s" % (p, g))
    
    # Armazenar a solução encontrada
    solucoes.append(solucao_atual)
    
    # Impedir que o solver encontre esta solução novamente
    bloqueio = []
    for p, g in solucao_atual:
        bloqueio.append(x[p][g])
    
    # Adiciona a negação da solução encontrada
    s.add(Not(And(bloqueio)))
    
    print("\n--- Nova distribuição ---\n")

if len(solucoes) == 0:
    print("Não tem solução.")
else:
    print(f"\nTotal de distribuições encontradas: {len(solucoes)}")


Ana fica no gabinete 1
Nuno fica no gabinete 1
Pedro fica no gabinete 3
Maria fica no gabinete 3

--- Nova distribuição ---

Ana fica no gabinete 2
Nuno fica no gabinete 2
Pedro fica no gabinete 3
Maria fica no gabinete 3

--- Nova distribuição ---

Ana fica no gabinete 3
Nuno fica no gabinete 2
Pedro fica no gabinete 3
Maria fica no gabinete 1

--- Nova distribuição ---

Ana fica no gabinete 1
Nuno fica no gabinete 2
Pedro fica no gabinete 3
Maria fica no gabinete 1

--- Nova distribuição ---

Ana fica no gabinete 2
Nuno fica no gabinete 2
Pedro fica no gabinete 3
Maria fica no gabinete 1

--- Nova distribuição ---

Ana fica no gabinete 2
Nuno fica no gabinete 1
Pedro fica no gabinete 3
Maria fica no gabinete 1

--- Nova distribuição ---

Ana fica no gabinete 3
Nuno fica no gabinete 1
Pedro fica no gabinete 3
Maria fica no gabinete 1

--- Nova distribuição ---

Ana fica no gabinete 1
Nuno fica no gabinete 2
Pedro fica no gabinete 1
Maria fica no gabinete 3

--- Nova distribuição ---



Use agora o SAT solver para testar a veracidade se cada uma das seguintes afirmações:

1. Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete.
2. Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete.

A que conclusões chegou?


In [4]:
from z3 import *

pessoas = ["Ana","Nuno","Pedro","Maria"]
gabs = [1,2,3]
x = {}
for p in pessoas:
    x[p] = {}
    for g in gabs:
        x[p][g] = Bool("%s,%d" % (p,g))

s = Solver()



# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
    s.add(Or([x[p][g] for g in gabs]))

# Cada pessoa ocupa no máximo um gabinete.
for p in pessoas:
    s.add(Sum([If(x[p][g], 1, 0) for g in gabs]) == 1)

# O Nuno e o Pedro não podem partilhar gabinete.
for g in gabs:
    s.add(Or(Not(x["Pedro"][g]), Not(x["Nuno"][g])))

# Se a Ana ficar sozinha num gabinete, então o Pedro também terá
# que ficar sozinho num gabinete.
#s.add(And((Sum([If(x["Ana"][g], 1, 0) for g in gabs]) == 1),(Sum([If(x["Pedro"][g], 1, 0) for g in gabs]) == 1))) #Não sei se está certa
for g in gabs:
    # Ana is in office g and no one else is in office g
    ana_alone = And(x["Ana"][g], Sum([If(x[p][g], 1, 0) for p in pessoas if p != "Ana"]) == 0)
    # Pedro is alone in some office
    pedro_alone = Or([And(x["Pedro"][g2], Sum([If(x[p][g2], 1, 0) for p in pessoas if p != "Pedro"]) == 0) for g2 in gabs])
    # If Ana is alone, then Pedro must be alone
    s.add(Implies(ana_alone, pedro_alone))

# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
for g in gabs:
    s.add(Sum([If(x[p][g], 1, 0) for p in pessoas]) <= 2)

# Afirmação 1. Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete.
af1 = Implies(x["Maria"][1], Sum([If(x[p][1], 1, 0) for p in pessoas if p != "Maria"]) == 0)

s.push()
s.add(Not(af1))
if s.check() == sat:
    print("Afirmação 1 é falsa. É possível que a Maria ocupe o gabinete 1 sem estar sozinha")
else:
    print("Afirmação 1 é verdadeira. Se a Maria ocupar o gabinete 1, ela ficará sozinha") 
s.pop() 

# Afirmação 2. Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete.
af2_1 = Or([And(x["Ana"][g], x["Nuno"][g]) for g in gabs])
af2_2 = Or([And(x["Maria"][g], x["Pedro"][g]) for g in gabs])
af2 = Implies(af2_1, af2_2)

s.push()
s.add(Not(af2))
if s.check() == sat:
    print("Afirmação 2 é falsa. Se a Ana e o Nuno ficarem no mesmo gabinete. É possível que a Maria e o Pedro ocupem um gabinete diferente")
else:
    print("Afirmação 2 é verdadeira. Se a Ana e o Nuno ficarem no mesmo gabinete, a Maria e o Pedro terão que partilhar um gabinete") 
s.pop()

if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Afirmação 1 é falsa. É possível que a Maria ocupe o gabinete 1 sem estar sozinha
Afirmação 2 é falsa. Se a Ana e o Nuno ficarem no mesmo gabinete. É possível que a Maria e o Pedro ocupem um gabinete diferente
Ana fica no gabinete 3
Nuno fica no gabinete 3
Pedro fica no gabinete 1
Maria fica no gabinete 2
